# Introduction to regular expressions with Python

---
Nicholas A. Lester


This file will 
- introduce the basics of regular expressions
- show how they are implemented in Python
- show how they can be applied to textual analysis using the `re` library 

There is much more to regex than is covered here. To learn more, visit the documentation for the `re` library [here](https://docs.python.org/3/library/re.html).

## What are regular expressions?


---


Regular expressions, or **regex**, comprise a formal language for matching (sub)strings. 

Regex are formally **finite state automata**, or **FSAs**. FSAs encode transitions between states (e.g., transitions between characters in a string of text). 

A simple FSA is presented below:

<center><img src="https://github.com/nalester/LING_5405/blob/main/Fig_fsa.png?raw=true", width=300></center>


In this representation, 
- the circles indicate states
= the arrows show the possibilities for moving between states
- the double-circle indicates a possible ending state

To see what strings it can match, simply follow the arrows until you reach the double-circle. The b cell has an optional repeating arrow, making this a *non-deterministic* FSA or NFSA.

As the figure shows, this FSA will match any sequence that consists of 
- exacrtly one *a*
- followed by any number of *b*
- followed by exactly one *c* 

Other strings, such as a single *a*, or any sequence that goes against the flow of the arrows (e.g., *bac*) will not be matched. 


---

#### **An example**
To demonstrate why this is useful, imagine that the above FSA is your search term. You can now move through a string or document one character at a time. The first *a* that you find is an initial starting point for the match. If it is not followed by *b*, keep moving. If it is followed by *b*, then we can still entertain the potential match, and the process continues. 

**Sample string**  
*aababbbccbabcb*

**Moving through the example on character at a time:**
- <font color="green">green</font> marks the starting character for a possible matching sequence along with any following characters that continue to be consistent
- <font color="darkorange">orange</font> marks ends of matching sequences
- <font color="lightgrey">grey</font> marks incompatible substrings

<center>

<font color="green">a</font>ababbbbccbabcb  

<font color="lightgrey">a</font><font color="green">ab</font>abbbbccbabcb  

<font color="lightgrey">aab</font><font color="green">a</font>bbbbccbabcb  

<font color="lightgrey">aab</font><font color="green">abb</font>bbccbabcb  

<font color="lightgrey">aab</font><font color="green">abbb</font>bccbabcb  

<font color="lightgrey">aab</font><font color="green">abbbb</font>ccbabcb  

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font>cbabcb

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">c</font>babcb         
<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">cb</font>abcb  

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">cb</font><font color="green">a</font>bcb  

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">cb</font><font color="green">ab</font>cb 

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">cb</font><font color="green">ab</font><font color="darkorange">c</font>b

<font color="lightgrey">aab</font><font color="green">abbbb</font><font color="darkorange">c</font><font color="lightgrey">cb</font><font color="green">ab</font><font color="darkorange">c</font><font color="lightgrey">b</font>

</center>

Thus, in the total string, we have two substrings that are compatible with the FSA defined above: **abbbbc** (starting at character 3 in 0-initialized counting) and **abc** (starting at character 11). 

The matching substrings can then be "clipped out" (either because we want to keep them or discard them). 

This example should hint at the possibility for **searching and segmenting text**. With more interesting regex, we can search for *any* substring, including words, sentences, and so on. 

All we need to know is how the words, sentences, etc. are demarcated in the document and what they can consist of internally. 

**A quick note**: the method of search through the string is independent of the regex. Here, we see a **greedy** approach. 

**Greedy matching** grabs the longest substring that is compatible with the search term. Only once the "furthest" possible endpoint has been reached is the next character considered a possible start for the next substring. 

## Regex Notation

---

**How do we "write out" an FSA like in the example above?**

Regex come with several types of special characters, in addition to the standard alphanumeric and punctuation set that we use on a daily basis (and which will comprise the majority of any human-language text). 

To start, regex are traditionally indicated by <font color="darkred">//</font>.

Example: <font color="darkred">/some_regex/</font> 

**NB**: This is NOT how regex are written in Python; we will cover that below.

#### **Identity matching**


---
You can match any standard character simply by typing it. These matches are case-sensitive.

**Example**  
- <font color="darkred">/cat/</font> matches *cat* but not *Cat*

Numbers work the same way.

Some punctuation marks need to be combined with a special character -- the **escape** character <font color="darkred">/\\/</font> in order to be interpreted as true punctuation.

This is because they are used on their own as special characters in regex. 

For example, the period "." is a special character. If you want to search for a string that contains an actual period, you must combine it with the escape character to form <font color="darkred">/\\./</font>.

<br>

**Disjunction** 


---


You can specify alternative identity matches using *disjunction*. 

The disjunction character is the pipe <font color="darkred">/|/</font>. It is best to combine the pipe with parentheses to demarcate the limits of the options.

<br>

**Example**  
- <font color="darkred">/(cat|dog|mouse)/</font> matches exactly *cat* **OR** *dog* **OR** *mouse*. 

- It **does not** match *ca* or *ous* or any other substring from among the alternatives.

#### **Positional matching**


---

You can specify where you want to look for a match relative to the beginning or ending of strings. 

- <font color="darkred">/^/</font> matches the beginning of a string
- <font color="darkred">/$/</font>  matches the ending of a string

**Examples**  
- <font color="darkred">/^word/</font> matches any string that **starts with** the character string "word". 

- <font color="darkred">/word$/</font> matches any string that **ends with** the character string "word".

- <font color="darkred">/^word$/</font> matches any string that **is exactly** the character string "word".

#### **Set matching**


---

We can also match characters from a set of possibilities.

Say **we don't know ahead of time how each string will look**, but we know that we want it to be composed only from certain characters, possibly in different combinations.

Or say **we don't care about the specific characters at all**, but only that *some* character or characters are there.

How can we code what we don't know ahead of time?

We define a **set**!

<br>

**Types of sets**


---


- <font color="darkred">/./</font>  
  **Wildcard**: stands for the set of all possible characters. Matches any character.

- <font color="darkred">/[abc]/</font>  
  **Character set**: matches **any one** character from the characters in the square brackets (here, *a* **OR** *b* **OR** *c*).   

- <font color="darkred">/[A-Za-z0-9]/</font>  
  **Character range**: some special sets are defined as ranges. For example, <font color="darkred">/[A-Z]/</font> matches any uppercase latin letter from *A* to *Z*. It is therefore equivalent to <font color="darkred">/[ABCDEFGHIJKLMNOPQRSTUVWXYZ]/</font>. The same logic applies to <font color="darkred">/[a-z]/</font> for lowercase latin letters and <font color="darkred">/0-9/</font> for numerals.

- <font color="darkred">/\\*x*/</font>   
  **Metacharacters**: more special sets. The general convention for **set-oriented metacharacters** is that lowercase *x* is a positive match ("look for this"), and uppercase *X* is a negative match ("look for anything that is not this").
  - <font color="darkred">/\d/</font>: digits
  - <font color="darkred">/\D/</font>: non-digits
  - <font color="darkred">/\w/</font>: word characters (= <font color="darkred">/[A-Za-z0-9_]/</font>)
  - <font color="darkred">/\W/</font>: non-word characters   

<br>

**Negating any set manually**


---


Finally, for any set defined in square brackets, e.g., <font color="darkred">/[abc]/</font>, you can manually say "not one of these" using <font color="darkred">/[^...]/</font>.
  - <font color="darkred">/[^abc]/</font> means "**NOT** (*a* **OR** *b* **OR** *c*)"
  - <font color="darkred">/[^\w]/</font> means <font color="darkred">/\W/</font>

#### **Quantifiers**


---

Quantifiers allow us to say **how many times a character or substring may be repeated** (like the circular arrow in the FSA above).

They are placed immediately after the unit that they control.

There are two basic types
- **special characters**
- **brace notation** 

<br>

**Special characters**


---

The special characters are <font color="darkred">/*/</font>, <font color="darkred">/+/</font>, and <font color="darkred">/?/</font>.

- <font color="darkred">/<i>x</i>\*/</font>  
  Matches 0 or more *x*

- <font color="darkred">/<i>x</i>\+/</font>  
  Matches 1 or more *x*

- <font color="darkred">/<i>x</i>\?/</font>  
  Matches 0 or 1 *x*

**Examples**
- <font color="darkred">/a\*/</font> matches nothing (empty string, i.e., 0 *a*) **OR** *a*, *aa*, *aaaaaaaaaaaa*, etc.

- <font color="darkred">/a\+/</font> matches *a*, *aa*, *aaaaaaaaaaaa*, etc. 

- <font color="darkred">/a\?/</font> matches nothing **OR** *a*.

<br>

**Brace notation**


---

You can customize the number and/or range of repetitions directly using curly braces <font color="darkred">/{*x*, *y*}/</font>, where *x* and *y* are any number.

- <font color="darkred">/a{4}/</font>  
  Matches **exactly 4** instances of *a*
  - *aaaa*

- <font color="darkred">/a{,4}/</font>  
  Matches 0 **up to** 4 instances of *a*, or *aaaa*
  - nothing **OR** *a*, *aa*, *aaa*, *aaaa*

- <font color="darkred">/a{4,}/</font>  
  Matches **4 or more** instances of *a*
  - *aaaa*, *aaaaa*, ..., *aaaaaaaaaaa*, etc.

- <font color="darkred">/a{4, 6}/</font>  
  Matches **from 4 to 6** instances of *a*
  - *aaaa*, *aaaaa*, *aaaaaa*

<br>

**Repeating longer substrings**


---

You can repeat longer substrings by applying the quantifier to a unit in parentheses: <font color="darkred">/(...)QUANT/</font> 

- <font color="darkred">/(cat)\+/</font> matches *cat*, *catcat*, *catcatcatcatcat*, etc. 

- <font color="darkred">/(cat){2, 5}/</font> matches *catcat*, *catcatcat*, *catcatcatcat*, *catcatcatcatacat*

#### **Other useful metacharacters**


---

These metacharacters refer to individual characters, rather than sets. 

- <font color="darkred">/\b/</font>: word boundary

  This is useful when you need to ensure that you match whole words (rather than substrings). It also falls between words and non-word-internal punctuation (period, comma, etc.). Thus, you can use it to make sure that you don't get hits like "car," or "there!" when you only want the word itself.
  
  **Example**  
  If you want to match the word *ate* only (and not, say, the sequence *ate* in the word *infl<b>ate</b>*), you can specify <font color="darkred">/\bate\b/</font>

- <font color="darkred">/\t/</font>: tab-stop

  The tab-stop is useful when you want to add or remove tabs. For example, one common way to organize data into columns is to separate each value by inserting a tab between them. Or you may want to change a tab-separated dataset into a comma-separated dataset by exchanging the tabs for commas.  

- <font color="darkred">/\n/</font>: new line character

  Marks the end of a line. Sometimes you need to clean these out of a text during initial processing. Or you may want to change the organization of a text visually by inserting new lines. Finally, you can use them to segment texts at the level of lines, however encoded (e.g., each line might be one sentence, and you want a list of the sentences). 

- <font color="darkred">/\r/</font>: carriage return

  Similar to new line. Some systems use a combination of new line and carriage return to signify the end of lines (e.g., Windoze). Usually, you are trying to get rid of them, or use them to segment a document. 

- <font color="darkred">/\s/</font>: white space

- <font color="darkred">/\S/</font>: non-white space

#### **Back to the FSA above**


---

We now have the tools to **rewrite the FSA above as a regex**.

We need 
- exactly one *a*
- followed by one or more *b*
- followed by exactly one *c*

or...

<font color="darkred">/ab+c/</font> 

## Implementing regex in Python

To use regex in Python, we need the `re` library. 

In [ ]:
import re

The `re` library comes with a number of **functions that employ regular expressions for string matching**. 

Some are improved versions of functions we already know. For example, `re.split` functions similarly to `split`, but allows you to state the splitting character as a regex.

Regex strings are encoded similar to standard strings, except that the opening quotation mark is preceded by an r. 
- **standard string**: `"string"`
- **regex**: `r"string"`

In [ ]:
string = "string"
print(string)

regex = r"regex"
print(regex)

The regex can be elaborated in all the ways described above (without the "//"), which can be interpreted appropriately in the functions from `re`.

## `re` functions
Here are a few useful functions
- `re.search()`
- `re.match()`
- `re.fullmatch()`
- `re.findall()`
- `re.split()`
- `re.sub()`

Some of these functions return `match` objects, which need to be "unpacked."

Before we get into the explanations of these functions, let's grab some data. 

We read in a an excerpt of Emma as a string and explore its contents.

In [ ]:
# This bit is specfic to loading data into google colab. It is also specific to my set-up. Adjust accordingly for the location of this file in your own system.

from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/austen-emma-excerpt.txt'

In [ ]:
with open(path, "r") as f:
  text = f.read()
  f.close()

In [ ]:
# Make sure we got what we wanted
print(type(text))

In [ ]:
# Have a peek
print(text[:20])

In [ ]:
print(text[-20:])

In [ ]:
# Explore types of characters (we will have to think about these when we write regex)
character_set = set(list(text))
print(character_set)

In [ ]:
#...with normalization
norm_character_set = set(list(text.lower()))
print(norm_character_set)

#### `re.search`

This function takes two arguments: 
- *pattern* -- the regex
- *string* -- the string to be searched

It looks for the **first matching substring** in *string* according to the regex *pattern*.  

If there is a match, it returns a `match` object. Use `.group(0)` to obtain that match. 

If there is no match, it returns `None`. 

Finally, you can use the `match` object returned by `re.search` as a Boolean (True/False) in conditions. If the substring is matched, the `match` object is treated as `True`; it is `False` otherwise. 


Let's start by looking for the word "man".

In [ ]:
regex = r"\bthe\b" # using word boundaries to make sure we ONLY get "man" 

result = re.search(regex, text)

print(result) # tells us that it is a match object, and also the indices at which it was found (the "span" bit -- t = 163, h = 164, e = 165)

print(result.group(0)) # access the information stored in the match object, i.e., return the matching substring

Note that there is only one match. That is because all `re.search` cares about is **the first match**.

Now we use the result of `re.search` as a condition in an `if` statement.

In [ ]:
test = "I am a text."

# A substring that IS in there
if re.search("text", test):
  print("It's in there!")
else:
  print("Nope!")

# A substring that is not in there
if re.search("Godzilla", test):
  print("It's in there!")
else:
  print("Nope!")

#### `re.match`

This function takes two arguments: 
- *pattern* -- the regex
- *string* -- the string to be searched

It looks for a pattern matching the regex **only at the beginning of the string**.  

If there is a match, it returns a `match` object. Use `.group(0)` to obtain that match. 

If there is no match, it returns `None`. 


In [ ]:
regex = r"\bthe\b" # using word boundaries to make sure we ONLY get "man" 

result = re.match(regex, text)

print(result) # no match now (unlike search)

regex = text[:10] # the actual first 10 characters

result = re.match(regex, text)

print(result)

print(result.group(0))

You can also use the ouput of `re.match` as a condition.

Below we recycle the code from above, but replace `.search` with `.match`.

In [ ]:
test = "I am a text."

# A substring that IS in there
if re.match("I am", test): # Remember: match only cares about the very beginning of the string
  print("It's in there!")
else:
  print("Nope!")

if re.match("text", test): #... so now this doesn't match.
  print("It's in there!")
else:
  print("Nope!")

# A substring that is not in there
if re.match("Godzilla", test):
  print("It's in there!")
else:
  print("Nope!")

#### `re.fullmatch`

This function takes two arguments: 
- *pattern* -- the regex
- *string* -- the string to be searched

It checks whether **the whole string matches the regex**. 

Otherwise, it is the same as `re.match`

In [ ]:
# Looking for a word
regex = r"\bthe\b" # using word boundaries to make sure we ONLY get "man" 

result = re.fullmatch(regex, text)

print(result) # no match now (unlike search)

# Looking just at the beginning of the string
regex = text[:10]

result = re.fullmatch(regex, text)

print(result) # no match now (unlike match; we didn't match the ENTIRE string)

# Now we test the whole string against itself
result = re.fullmatch(text, text)

print(result.group(0))

**Now for some more interesting functions!**

#### `re.findall`

This function takes two arguments: 
- *pattern* -- the regex
- *string* -- the string to be searched

It searches (**greedy!**) through the entire string and **returns all matches** to the regex in a list. 

If there is no match, it returns an empty list. 

**This is one of the most useful tools for collecting word- or broader-level tokens from a string.** 

In [ ]:
# Start simple looking for all the's
regex = r"\bthe\b"

result = re.findall(regex, text)

print(result)

# Look for a word that isn't there
regex = r"\bGodzilla\b"

result = re.findall(regex, text)

print(result)

But we can get some more interesting kinds of matches.

In [ ]:
# All four-letter words?
regex = r"\b\w{4}\b"

result = re.findall(regex, text)

print(set(result))

In [ ]:
# All words ending in -ed? (a proxy for past-tense verbs)
regex = r"\w+ed\b"

result = re.findall(regex, text)

print(set(result))

In [ ]:
# Proper nouns? (capitalized words not at the beginning of sentences)
regex = r" [A-Z]\w+\b" 

result = re.findall(regex, text)

print(set(result))

We needed the whitespace at the front to find sentence-medial capitalized words. But it also shows up in our matches -- yuck!

We can fix this using parentheses in the regex. `re.findall` will return the same matches, but only the substring from the match that falls within the parentheses. 

In [ ]:
# Proper nouns without whitespace at front?
regex = r" ([A-Z]\w+)\b" # Note the use of parentheses -- with them, we can use the whitespace TO match, but NOT INCLUDE it in the resulting match.

result = re.findall(regex, text)

print(set(result))

#### `re.split`

This function takes two arguments: 
- *split* -- the regex specifying split character(s)
- *string* -- the string to be searched

Splits are made at any substring matching the regex. 

It returns a list of strings, where each element was separated from the others by the split terms. The split term is not included in what is returned. 

In [ ]:
# Split at whitespace or any punctuation or line endings
regex = r"[ \{\}\(\)\[\]\,\.\!\?\n\;]+" # I picked this set based on the earlier exploration of the data.

result = re.split(regex, text)

print(result)

A simple English **word tokenizer**!

What if we want sentences?

In [ ]:
# Split at sentence-final punctuation (with trailing space or new line).
regex = r"[\!\?\.][ \n]+"

result = re.split(regex, text[48:])

print(result)

A simple English **sentence tokenizer**!

Note that there is still a bunch of rubbish in there, such as sentence-medial new-line characters. 

We can also use regular expressions to clean up the text.

#### `re.sub`

This function takes three arguments: 
- *pattern* -- the regex specifying what you want to BE replaced
- *repl* -- the string you want to do the replacing
- *string* -- the string to be searched

Searches the string from the beginning (**greedy**) and **replaces each match to the regex with the string in *repl***.

Returns the whole string with the substitutions in place.


In [ ]:
# Replace those pesky new line characters and extra spaces with a single space
regex = r"(\n+| {2,})"

result = re.sub(regex, " ", text)

print(result)

In [ ]:
# Clean out punctuation
regex = r"[\{\}\(\)\[\]\,\.\!\?\;]+"

result = re.sub(regex, "", text)

print(result)

We can also do other kinds of replacements. Sometimes you want to reduce certain tokens to a class lab.

For example, we might not care what pronouns are present; only that there ARE pronouns present.  We could replace all pronouns with "PRONOUN"

In [ ]:
# Replace any pronoun with the label PRONOUN
regex = r"\b([Hh](is|im|er|ers)|[iI]ts?|[sS]he|[tT]he(ir|y|m)|[whmWHM]e|[oO]ur|[uU]s|I|mine|[Mm]y)\b"

result = re.sub(regex, "PRONOUN", text)

print(result)

Emma by Jane Austen 1816

VOLUME PRONOUN

CHAPTER PRONOUN


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex PRONOUN.

PRONOUN was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of PRONOUN sister's marriage,
been mistress of PRONOUN house from a very early period.  PRONOUN mother
had died too long ago for PRONOUN to have more than an indistinct
remembrance of PRONOUN caresses; and PRONOUN place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.


## Some practice exercises

---



1. Write a regex to match American telephone numbers.   
  
  Start by making a list of all the different forms a phone number can take:

   -  +1 940 555 5483
   -  1-940-555-5483
   -  (940) 555 5483
   - ...
  
  <font color="dodgerblue">Don't worry about capturing EVERY possible variation. Regex require a tradeoff between **recall** (getting what you want) and **effort** (how long are you willing to keep adding to a -- potentially giant -- regex to get all and only what you want?).</font>  

  Once you have it, test it against some different phone number formats to see if it matches what it should.

In [ ]:
# Phone number matcher

2. Write a regex to match all names with honorifics (Ms., Dr., Dame, Sir, Esq., M.D.,...).

  Remember that they can fall on either side of the name!

In [ ]:
# Honorable names matcher

3. Find all instances of -ly adverbs in Emma. Remember to include comparative (-ier) and superlative (-iest) forms. 

  Be careful with parentheses (should you choose to use them). They will be considered "capturing" parentheses and affect your output (only the content within the capturing parentheses will be returned, instead of the entire "proper" match). 
  
  If you want to use parentheses just for general regex purposes, use **<font color="orange">non-capturing parentheses</font>** ny adding **<font color="orange">?:</font>** to the right of the opening parenthesis. Like so:

  r"(<font color="orange"><b>?:</b></font>x|y|z)abc"

  Now they will work as expected.

In [ ]:
# This bit is specfic to loading data into google colab. It is also specific to my set-up. Adjust accordingly for the location of this file in your own system.
path = '/content/drive/My Drive/austen-emma.txt'

with open(path, "r") as f:
  full_text = f.read()
  f.close()

In [ ]:
# adverb matcher

4. How many **unique** (`set`) adverbs end in -ly? -ier? -iest?

In [ ]:
# counting adverbs with each ending

5. Extract potential verbs ending in -ing and -ed endings.

  Then, stem the list of verbs so that they no longer carry the ending -ing/-ed.
  
  *Stemming is the process of removing strings known to reflect inflectional morphology from the ends of words to arrive at an approximation of the word's root form.*


In [ ]:
# finding -ing and -ed verbs

Examine the results. Does it do a good job? Where does it do well? Where does it suffer?

In [ ]:
# examine results

(Summary of performance)